In [108]:
#!/usr/bin/env python
# coding: utf-8

import sys
import os
import matplotlib.pyplot as plt
import glob

from datetime import datetime
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox
# import keras
import pandas as pd
import numpy as np
import random
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from functools import partial

class interactive_data_chooser:
    """
    Class for selecting data graphically and displaying it
    """
    def __init__(self, df, columns):
        # we don't need this dataframe, make a df_copy instead?
        self.outlier_df = pd.DataFrame()

        # self.df = df
        self.df_copy = df.copy()
        self.columns = columns
        self.df_copy["manual_outlier"] = -1
        self.df_copy["model_outlier"] = 0

        self.axis_dropdowns = None
        self.chosen_color_column = self.df_copy["manual_outlier"]
    
    def activate_plot(self):
        """
        Display interactive plot where images (data points in the plot)
        can be selected using box select or lasso select. 
        """
        # TODO: cmin and cmax depending on chosen_color_column (manual_outlier will always be -1 to 1) 
        self.df_copy.reset_index(inplace=True,drop=True)
        numeric_df = self.df_copy.select_dtypes(include=np.number)
        numeric_columns = numeric_df.columns
        self.f = go.FigureWidget([go.Scatter(y = self.df_copy[self.columns[0]], x = self.df_copy[self.columns[1]], mode = 'markers',
                                       selected_marker_color = "red", 
                                             marker=dict(color=numeric_df[numeric_columns[0]], 
                                                        colorbar=dict(thickness=10), colorscale=["blue", "green", "orange"]))])
        
        scatter = self.f.data[0]
        scatter.marker.opacity = 0.5
        
        self.axis_dropdowns = interactive(self.update_axes, yaxis = self.columns, xaxis = self.columns, color = numeric_columns)
        # fig.for_each_trace(lambda trace: trace.on_selection(on_selection))
        scatter.on_selection(self.selection_fn)
        
        # Put everything together
        return VBox((HBox(self.axis_dropdowns.children),self.f))
    
    def update_axes(self, xaxis, yaxis,color):
        scatter = self.f.data[0]
        scatter.x = self.df_copy[xaxis]
        scatter.y = self.df_copy[yaxis]
        scatter.marker.color = self.df_copy[color]
        with self.f.batch_update():
            self.f.layout.xaxis.title = xaxis
            self.f.layout.yaxis.title = yaxis

    # def multiply_rows(row): Use this solution instead of iterrows
        # return row['column1'] * row['column2']

        # my_df['multiplied'] = my_df.apply(multiply_rows,axis=1)

    def update_temp_df_last_sel(self, row, last_selected):
        row["last_selected"] = last_selected
        return row
    
    def update_temp_df_man_ol(self, row):
        row["manual_outlier"] = 1 if self.df_copy.at[row[0], "manual_outlier"] != 1 else 0
        return row
    
    def update_df(self, row):
        row["manual_outlier"] = 1 if self.df_copy.at[row[0], "manual_outlier"] != 1 else 0
        return row
    
    def get_manual_outlier(row):
        return row
    
    def update_markers(self, trace, points, selector):  # Skip this?
        print("Inside update_markers()")
        marker = dict(symbol=["circle", "x", "triangle-up"],
                      symbolsrc="self.chosen_color_column")
        print("Before trace.marker.selected = marker")
        # trace.marker.selected = marker
        print(f"points_inds: {points.point_inds}")
        print(f"selector: {selector}")
        print(f"trace: {trace}")
        #trace.marker.symbol = 
        print("After trace.marker.selected = marker")
        # print(f"trace.marker.selected: {trace.marker.selected}")

    def map_colors_to_symbols(self, colors):
        symbols = []
        for color in colors:
            if color == 1:
                symbols.append("x")
            if color == 0:
                symbols.append("triangle-up")
            else:
                symbols.append("circle")
        return symbols
        

    def selection_fn(self,trace,points,selector):
        """
        Keeping track of points manually selected and change values in column ["manual_outlier"].
        Value for points not manually selected is -1. If selected to be an outlier, value is set to 1.
        If selected again not to be an outlier, value is set to 0. Previous value is stored for future 
        possibility to undo selection. 

        Each selection is stored in a temp_df and all temp_df's are stored in self.outlier_df.
        The dataframe drop_duplicates_df is the df which will be used to train the model, where only 
        the last manually made change to a data point is included. 

        The plot is updated after selection.
        """
        temp_df = self.df_copy.loc[points.point_inds]
        
        last_selected = len(temp_df)
        
        # my_df['multiplied'] = my_df.apply(multiply_rows,axis=1)
        # The method needs a variable, what to do?
        # temp_df["last_selected"] = temp_df.apply(self.update_temp_df_last_sel, axis=1)
        for i in temp_df.iterrows():
            idx = i[0]
            temp_df.at[idx, "last_selected"] = last_selected
            # This is needed for keeping track of the changes
            temp_df.at[idx, "manual_outlier"] = 1 if self.df_copy.at[idx, "manual_outlier"] != 1 else 0
            # This is needed for displaying values in the plot
            self.df_copy.at[idx, "manual_outlier"] = 1 if self.df_copy.at[idx, "manual_outlier"] != 1 else 0

        self.outlier_df = pd.concat([self.outlier_df, temp_df], ignore_index=False, axis=0)

        no_points = "point" if last_selected == 1 else "points"
        print(f"Selected {last_selected} new {no_points}. Total: {len(self.outlier_df)}")

        # Drop duplicates
        drop_duplicates_df = self.outlier_df.drop_duplicates(subset=["x", "y1"], keep="last")
        drop_duplicates_df.sort_values(by=["x"], inplace=True)
        print(f"Unique points selected ({len(drop_duplicates_df)}):")
        for i in drop_duplicates_df.iterrows():
            outlier = "yes" if i[1][3] == 1 else "no"
            print(f"x: {int(i[1][0])}, y1: {int(i[1][1])}, outlier: {outlier}")

        self.chosen_color_column = self.axis_dropdowns.children[2].value
        # Which indices are chosen?
        # Change marker symbol after select
        selected_indices = [p for p in temp_df.index.values]
        """ self.f.data[0].marker.symbolsrc="self.chosen_color_column"
        self.f.data[0].marker.symbol=["circle", "x", "triangle-up"] """

        print("Before on_selection")
        # print(self.f.data[0])
        # callback = partial(self.update_markers)
        # self.f.data[0].on_selection(callback)
        # trace.marker.symbolsrc = "trace.marker.color"
        # trace.marker.symbol = ["circle", "x", "triangle-up"]
        # self.f.data[0].on_selection(lambda self, trace: self.update_markers(self, trace))
        print(f"trace: {trace}")
        print("After on_selection")
        print(f"points: {points}")
        selected_points = [point for point in points.point_inds]
        symbols = self.map_colors_to_symbols(self.df_copy.loc[selected_points], "manual_outlier")
        self.f.update_traces(marker=dict(symbol=symbols, selector=dict(type="scatter")))
        
        
        # print(f"selected_indices: {selected_indices}")
        # print(f"temp_df: {temp_df}")
        # print(f"df_copy: {self.df_copy}")
        # Do I need selected points? And if so, how do I link them to df_copy? selectedpoints=selected_indices,
        """ self.f.update_traces(
                            marker=dict(symbol=[{"-1": "circle", "0": "triangle-up", "1": "x"}[label] for label in self.df_copy[self.chosen_color_column]])
        ) """
        """ self.f.update_traces(selectedpoints=selected_indices,
                            marker=dict(
                            symbol=[self.df_copy.loc[i, self.chosen_color_column] for i in selected_indices])
        ) """
    

        # Update color
        # trace.update(marker_color=self.df_copy[self.chosen_color_column])
        # print(self.f)
        # for i in trace.selectedpoints:
            # self.f.for_each_trace(
                # lambda trace: trace.update(marker_symbol="square") if trace.marker.color[i] == 1 else ())

    def clear_selection(self):
        self.outlier_df = self.outlier_df.iloc[0:0]
    
    def show_selected(self):
        for index, row in self.outlier_df.iterrows():
            plt.figure()
            plt.imshow(plt.imread(row['file']))
            plt.title(f"{row['time']}, wl: {row['wl']}, turb_s: {row['turb_sensor']}, turb_p: {row['turb_post']}")

    # create train model function based on outlier status in self.df

    # visualize result in graph

    # function to mark point as non-outlier DONE

    # button to undo choice

    # button to confirm (then train model), disable if not choosen areas == 1



In [109]:
#!/usr/bin/env python
# coding: utf-8

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

def create_fake_df(n):
    """
    Creates a dataframe with n rows and columns "x", "y1" and "y2". 
    The data are integers, 0-100.
    """
    x = []
    y1 = []
    y2 = []

    for _ in range(n):    
        x_int = random.randint(0, 100)
        x.append(x_int)
        y1_int = random.randint(0, 100)
        y1.append(y1_int)
        y2_int = random.randint(0, 100)
        y2.append(y2_int)

    int_dict = {"x": x, "y1": y1, "y2": y2}
    df = pd.DataFrame(int_dict)
    return df

df = create_fake_df(15)
chooser = interactive_data_chooser(df, df.columns)
chooser.activate_plot()


Selected 2 new points. Total: 2
Unique points selected (2):
x: 53, y1: 99, outlier: yes
x: 58, y1: 76, outlier: yes
Before on_selection
trace: Scatter({
    'marker': {'color': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int64),
               'colorbar': {'thickness': 10},
               'colorscale': [[0.0, 'blue'], [0.5, 'green'], [1.0, 'orange']],
               'opacity': 0.5},
    'mode': 'markers',
    'selected': {'marker': {'color': 'red'}},
    'selectedpoints': [0, 7],
    'uid': 'd4f68c53-e083-4146-b361-41e3d3cc868b',
    'x': array([53, 77, 21,  5, 93, 83,  0, 58, 22, 34, 13, 19, 82, 86, 66], dtype=int64),
    'y': array([99, 79, 78, 46, 91, 16, 13, 76, 98, 39, 90, 66, 28,  4,  2], dtype=int64)
})
After on_selection
points: Points(point_inds=[0, 7],
       xs=[53, 58],
       ys=[99, 76],
       trace_name='trace 0',
       trace_index=0)


TypeError: interactive_data_chooser.map_colors_to_symbols() takes 2 positional arguments but 3 were given

In [3]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create example data
import numpy as np
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)

# Create scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers'))

# Add dropdown menu
updatemenus = [{'buttons': [{'args': [{'selectedpoints': [0]}],
                              'label': 'Mean',
                              'method': 'restyle'},
                             {'args': [{'selectedpoints': [0]}],
                              'label': 'Median',
                              'method': 'restyle'}],
                'direction': 'down',
                'showactive': True}]
fig.update_layout(updatemenus=updatemenus)

# Add box select event handler
fig.add_layout_image(dict(
    source="https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/gitbook/images/lasso.png",
    xref="paper", yref="paper",
    x=1.05, y=1.2,
    sizex=0.2, sizey=0.2,
    xanchor="right", yanchor="bottom"
))

fig.update_layout(
    dragmode="select",
    annotations=[
        dict(
            text="Draw a box around the points to select",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.5, y=1.1, font_size=16
        )
    ]
)

fig.update_layout(
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title",
    title="Box Select and Dropdown Example"
)

# Define callback function for dropdown menu
def update_fig(selection):
    if selection == 'Mean':
        fig.update_traces(marker=dict(color='red', size=10))
        selected_points = fig.data[0].selectedpoints
        if selected_points:
            mean_value = np.mean(y[selected_points])
            fig.add_annotation(text=f"Mean: {mean_value:.2f}", xref="paper", yref="paper",
            x=0.8, y=0.9, showarrow=False, font_size=16)
    elif selection == 'Median':
        fig.update_traces(marker=dict(color='green', size=10))
        selected_points = fig.data[0].selectedpoints
        if selected_points:
            median_value = np.median(y[selected_points])
            fig.add_annotation(text=f"Median: {median_value:.2f}", xref="paper", yref="paper",
            x=0.8, y=0.8, showarrow=False, font_size=16)

# Set up callback for dropdown menu
fig.for_each_trace(lambda trace: trace.on_selection(lambda x,y,trace=trace: update_fig('Mean')))

fig.show()


In [2]:
import plotly.graph_objs as go
from ipywidgets import Button, Dropdown
from IPython.display import display

# Create example data
x = [1, 2, 3, 4, 5]
y = [10, 20, 30, 40, 50]

# Create plotly plot with box select enabled
trace = go.Scatter(x=x, y=y, mode='markers')
layout = go.Layout(title='Selected Points', xaxis=dict(range=[0, 6]), yaxis=dict(range=[0, 60]), dragmode='select')
fig = go.Figure(data=[trace], layout=layout)

# Define event handler function to display dropdown menu
def handle_selection(event):
    if event is None or len(event) == 0:
        return
    
    # Create the dropdown menu with the selected values
    dropdown_options = [
        {'label': 'Mean', 'value': 'mean'},
        {'label': 'Max', 'value': 'max'},
        {'label': 'Min', 'value': 'min'},
    ]
    dropdown = Dropdown(options=dropdown_options, description='Select an operation')
    display(dropdown)
    
# Add event handler function to the plotly plot
fig.layout.on_change(handle_selection, 'selectedpoints')

# Create a button to clear the selection and hide the dropdown menu
button = Button(description='Clear Selection')
button.on_click(lambda _: dropdown.close() if 'dropdown' in globals() else None)
display(fig, button)




ValueError: Invalid property specification(s): ['selectedpoints']

In [1]:
import plotly.graph_objs as go

fig = go.Figure(data=go.Scatter(x=[1, 2, 3, 4, 5],
                                y=[1, 2, 3, 4, 5],
                                mode='markers'))

fig.update_layout(
    dragmode='select',
    selectdirection='h',
    boxmode='group',
    # Add the box select tool
    selections=dict(
        mode='lasso',
        # Set the shape of the lasso selector to be a triangle
        shape='triangle'
    )
)

def update_selected_points(trace, points, selector):
    # Get the indices of the selected points
    inds = [p['pointIndex'] for p in points['points']]
    # Update the marker symbol of the selected points to triangles
    fig.data[0].marker.symbol[inds] = 'triangle'

fig.update_layout(
    dragmode='select',
    selectdirection='height',
    boxmode='group',
    selections=dict(
        mode='lasso',
        shape='triangle',
        # Add the callback function to the box select tool
        onselection=update_selected_points
    )
)




ValueError: 
    Invalid value of type 'builtins.dict' received for the 'selections' property of layout
        Received value: {'mode': 'lasso', 'shape': 'triangle'}

    The 'selections' property is a tuple of instances of
    Selection that may be specified as:
      - A list or tuple of instances of plotly.graph_objs.layout.Selection
      - A list or tuple of dicts of string/value properties that
        will be passed to the Selection constructor

        Supported dict properties:
            
            line
                :class:`plotly.graph_objects.layout.selection.L
                ine` instance or dict with compatible
                properties
            name
                When used in a template, named items are
                created in the output figure in addition to any
                items the figure already has in this array. You
                can modify these items in the output figure by
                making your own item with `templateitemname`
      

In [4]:
import plotly.graph_objs as go

fig = go.Figure(data=go.Scatter(x=[1, 2, 3, 4, 5],
                                y=[1, 2, 3, 4, 5],
                                mode='markers'))

fig.update_layout(
    dragmode='select',
    selectdirection='h',
    boxmode='group',
    # Add the box select tool
    **dict(
        select=dict(
            mode='lasso',
            # Set the shape of the lasso selector to be a triangle
            shape='triangle'
        )
    )
)

def update_selected_points(trace, points, selector):
    # Get the indices of the selected points
    inds = [p['pointIndex'] for p in points['points']]
    # Update the marker symbol of the selected points to triangles
    fig.data[0].marker.symbol[inds] = 'triangle'

fig.update_layout(
    dragmode='select',
    selectdirection='height',
    boxmode='group',
    **dict(
        select=dict(
            mode='lasso',
            shape='triangle',
            # Add the callback function to the box select tool
            onselection=update_selected_points
        )
    )
)


ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'select'

Did you mean "height"?

    Valid properties:
        activeselection
            :class:`plotly.graph_objects.layout.Activeselection`
            instance or dict with compatible properties
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regard

In [49]:
import plotly.graph_objs as go

# Create some data
x = [1, 2, 3, 4, 5]
y = [1, 4, 9, 16, 25]

# Create a FigureWidget object with a scatter plot
fig = go.FigureWidget(data=[go.Scatter(x=x, y=y, mode='markers')])

# Define a callback function that updates the trace
def update_markers(trace, points, selector):
    # Create a new marker style for the selected points
    marker = dict(color='red', size=10)
    # Update the trace's selected marker properties
    trace.marker.selected = marker

# Register the callback function with the scatter plot
fig.data[0].on_selection(update_markers)

# Display the FigureWidget
fig


FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': 'af947a45-13e1-4311-bd0a-6abadcf05294',
              'x': [1, 2, 3, 4, 5],
              'y': [1, 4, 9, 16, 25]}],
    'layout': {'template': '...'}
})

ValueError: Invalid property specified for object of type plotly.graph_objs.scatter.Marker: 'selected'

Did you mean "sizeref"?

    Valid properties:
        angle
            Sets the marker angle in respect to `angleref`.
        angleref
            Sets the reference for marker angle. With "previous",
            angle 0 points along the line from the previous point
            to this one. With "up", angle 0 points toward the top
            of the screen.
        anglesrc
            Sets the source reference on Chart Studio Cloud for
            `angle`.
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `marker.colorscale`. Has an effect only if in
            `marker.color` is set to a numerical array. In case
            `colorscale` is unspecified or `autocolorscale` is
            true, the default palette will be chosen according to
            whether numb

ValueError: Invalid property specified for object of type plotly.graph_objs.scatter.Marker: 'selected'

Did you mean "sizeref"?

    Valid properties:
        angle
            Sets the marker angle in respect to `angleref`.
        angleref
            Sets the reference for marker angle. With "previous",
            angle 0 points along the line from the previous point
            to this one. With "up", angle 0 points toward the top
            of the screen.
        anglesrc
            Sets the source reference on Chart Studio Cloud for
            `angle`.
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `marker.colorscale`. Has an effect only if in
            `marker.color` is set to a numerical array. In case
            `colorscale` is unspecified or `autocolorscale` is
            true, the default palette will be chosen according to
            whether numb

In [7]:
import plotly.graph_objs as go

fig = go.Figure(data=go.Scatter(x=[1, 2, 3, 4, 5],
                                y=[1, 2, 3, 4, 5],
                                mode='markers'))

fig.update_layout(
    yaxis=dict(domain=[0.1, 1]),
    yaxis2=dict(domain=[0, 0.1]),
    dragmode='select',
    selectdirection='h',
    boxmode='group',
    # Add the box select tool
    selections=dict(
        mode='lasso',
        # Set the shape of the lasso selector to be a triangle
        shape='triangle'
    )
)

def update_selected_points(trace, points, selector):
    # Get the indices of the selected points
    inds = [p['pointIndex'] for p in points['points']]
    # Update the marker symbol of the selected points to triangles
    fig.data[0].marker.symbol[inds] = 'triangle'

fig.update_layout(
    dragmode='select',
    selectdirection='h',
    boxmode='group',
    selections=dict(
        mode='lasso',
        shape='triangle',
        # Add the callback function to the box select tool
        onselection=update_selected_points
    )
)


ValueError: 
    Invalid value of type 'builtins.dict' received for the 'selections' property of layout
        Received value: {'mode': 'lasso', 'shape': 'triangle'}

    The 'selections' property is a tuple of instances of
    Selection that may be specified as:
      - A list or tuple of instances of plotly.graph_objs.layout.Selection
      - A list or tuple of dicts of string/value properties that
        will be passed to the Selection constructor

        Supported dict properties:
            
            line
                :class:`plotly.graph_objects.layout.selection.L
                ine` instance or dict with compatible
                properties
            name
                When used in a template, named items are
                created in the output figure in addition to any
                items the figure already has in this array. You
                can modify these items in the output figure by
                making your own item with `templateitemname`
      

In [5]:
glen_1 = pd.read_csv("data/asset-data-export_O12QnL6kAl-640876dfe5066-1678276319.csv")
glen_1

,"Asset name;""NU4202 Naset"""
"Exported on;""8 March 2023","12:51:59"""
"Timezone;""UTC +1""",NaN
"Date/Time;""Water level","Nap (cm)"""
2022-03-08 00:00:04;24,NaN
2022-03-08 00:30:00;24,NaN
...,...
2023-03-07 23:00:00;26,NaN
2023-03-07 23:30:00;25,NaN
2023-03-08 00:00:04;25,NaN
2023-03-08 00:30:00;25,NaN


In [6]:
glen_2 = pd.read_csv("data/asset-data-export_Zga3AM63oO-64098cbc36e23-1678347452.csv")
glen_2

,Asset name;NU4201 Spektrumgatan
Exported on;9 March 2023,08:37:32
Timezone;UTC +1,NaN
;,NaN
Date/Time;Water level,Nap (cm)
2022-03-09 00:00;5,NaN
...,...
2023-03-08 23:00;1,NaN
2023-03-08 23:30;1,NaN
2023-03-09 00:00;1,NaN
2023-03-09 00:30;1,NaN


In [12]:
glen_3 = pd.read_csv("data/asset-data-export_vEm3Jd5916-64098d190a5ca-1678347545.csv")
glen_3.head()

,Asset name;NU3328 Strandvägen
Exported on;9 March 2023,08:39:05
Timezone;UTC +1,NaN
;,NaN
Date/Time;Water level,Nap (cm)
2022-04-22 12:47;11,NaN
